In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)

        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
myNet = Net()
print(myNet)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [ ]:
params = list(myNet.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [ ]:
inputs = torch.randn(1, 1, 32, 32)
outputs = myNet(inputs)
print(outputs)

tensor([[ 0.0190, -0.1068,  0.0496,  0.0935,  0.0276, -0.1556,  0.0935,  0.0313,
          0.1177, -0.1891]], grad_fn=<AddmmBackward0>)


In [ ]:
myNet.zero_grad()
outputs.backward(torch.randn(1, 10))

In [ ]:
outputs = myNet(inputs)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()
loss = criterion(outputs, target)
print(loss)
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

tensor(1.1440, grad_fn=<MseLossBackward0>)


In [ ]:
myNet.zero_grad()
print(myNet.conv1.bias.grad)
loss.backward()
print(myNet.conv1.bias.grad)

tensor([0., 0., 0., 0., 0., 0.])
tensor([-0.0069,  0.0098, -0.0085,  0.0291, -0.0147, -0.0066])


In [ ]:
learning_rate = 0.01
for f in myNet.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(myNet.parameters(), lr=0.01)
optimizer.zero_grad()
outputs = myNet(inputs)
loss = criterion(outputs, target)
loss.backward()
optimizer.step()